# Setting Up Your Local System

## Introduction

CMB-ML manages a complex pipeline that processes data across multiple stages. Each stage produces outputs that need to be tracked, reused, and processed in later stages. Without a clear framework, this can lead to disorganized code, redundant logic, and errors.

The CMB-ML library provides a set of tools to manage the pipeline in a modular and scalable way. 

This notebook guides you through the essential setup required to run CMB-ML locally. File locations need to be set, and critical external assets are needed. Without completing these steps, the pipeline -- and subsequent demos -- will be missing dependencies and report bad file paths.

The previous demonstration, [describing Hydra](./A_hydra_tutorial.ipynb), was a tutorial. This notebook is a practical checklist, getting everything in place. It can be skipped if you're just looking through the repo, but will be needed if you want to run any code.

The rest of the demonstrations are tutorials.

Note that this is a work in progress. To further support modular code, CMB-ML has been set up as an installable package and includes config files. I've tried to update this and other notebooks, but if you see something that doesn't quite make sense, it may be an issue on my end. Please let me know!

## Contents

View this notebook with [nbviewer](https://nbviewer.org/github/CMB-ML/cmb-ml/tree/main/demonstrations/C_setting_up_local.ipynb#Introduction) to enable these links.

- [Set up the configuration](#Setting-configurations)
- [Set up PyILC](#Setting-up-PyILC)
- [Download external science assets](#Getting-science-assets)
- [Get the dataset](#Getting-the-dataset)
- [Next steps](#Next-steps)

# Setting configurations

## Your local system

First you'll need to tell the system where large data files should be stored. For flexibility, this is handled within the configuration files.

You'll need to create whatever target folder you want to use. This is a deliberate choice to prevent accidental duplication of data.

The default configuration relies on setting an environment variable that specifies a root directory for CMB-ML (this is the suggested method). Suppose you want this to be in your home directory, e.g. "~/CMB-ML_data/"; you would need to run `export CMB_ML_DATA=~/CMB-ML_data/` before running any scripts. You may want to add this line to your shell startup script (e.g., `.bashrc` or `.szhrc`). When running notebooks, use
```python
import os
os.environ['CMB_ML_DATA'] = "~/CMB-ML_data"
```
You'll want to change this in notebooks as you go (it is currently set with `os.environ['CMB_ML_DATA'] = '/data/jim/CMB_Data'`).

***Alternatively***, you may change the CMB-ML configuration file itself. First, find where it is installed (using `pip show cmbml` or `python -c "import cmbml; print(cmbml.__path__)"`). Within the installed `cmbml` directory, go to `/cfg/local_system` and open `generic_lab.yaml`. Edit `datasets_root` and `assets_dir` for your system. For instance, if you want to install to `'/data/jim/CMB_Data'`, you could use:
```yaml
datasets_root    : "/data/jim/CMB_Data/Datasets/"
assets_dir       : "/data/jim/CMB_Data/Assets/"
```

or your home directory could be set up with a hidden folder using:
```yaml
datasets_root    : "~/.CMB-ML/Datasets/"
assets_dir       : "~/.CMB-ML/Assets/"
```

## Checking the configuration

You should now have set both your local system configuration and [config_setup.yaml](../cfg/config_setup.yaml). 

The following two code cells will confirm your setup. If you aren't using an environment variable, you can remove that line. If you are using the environment variable, change its value.

In [ ]:
import os
from pathlib import Path
import hydra
from hydra import compose, initialize
from omegaconf import OmegaConf

# Set the environment variable, only effective for this notebook.
# Remove the next line if preferred
os.environ['CMB_ML_DATA'] = '/data/jim/CMB_Data'

In [2]:
# Clear any previous hydra instance to prevent conflicts
hydra.core.global_hydra.GlobalHydra.instance().clear()

# Initialize hydra with the configuration directory
with initialize(version_base=None):
    cfg = compose(config_name='config_setup.yaml',
                  overrides=["~file_system"])

print(OmegaConf.to_yaml(cfg))

print()
ds_root = Path(cfg.local_system.datasets_root)
assets_root = Path(cfg.local_system.assets_dir)
print(f"{ds_root} exists: {ds_root.exists()}")
print(f"{assets_root} exists: {assets_root.exists()}")

local_system:
  datasets_root: ${oc.env:CMB_ML_DATA}/Datasets/
  assets_dir: ${oc.env:CMB_ML_DATA}/Assets/


/data/jim/CMB_Data/Datasets exists: True
/data/jim/CMB_Data/Assets exists: True


If everything is set up correctly, the printed paths should match what you expect. If not, double-check your YAML and environment variables.

If the directories do not exist, they should be created.

In [ ]:
# Uncomment the next lines to create those directories
# ds_root.mkdir(parents=True, exist_ok=True)
# assets_root.mkdir(parents=True, exist_ok=True)

# Setting up PyILC

PyILC is used as a baseline method for cleaning observation maps. Please check out their work at [PyILC on GitHub](https://github.com/jcolinhill/pyilc). As of December, 2024, PyILC isn't structured as an installable library. I've settled on a workaround of importing the necessary elements in a CMB-ML module so it can be used without modification or unnecessary duplication of effort.

This may not be ideal and I'm open to feedback.

## Step 1: Clone PyILC
Skip ahead if you're comfortable with this.

Navigate to where you want to put the PyILC code. Assuming your file structure is something like
```
└─ home
   └─ code
      ├─ cmb-ml
      |     └─ all this stuff
      ├─ other-repo
```
I suggest:
```bash
cd /home/code
git clone https://github.com/jcolinhill/pyilc.git 
```
which will install it where "other-repo" is. I do not suggest installing it within `home/code/cmb-ml` to prevent confusion with `.git`.

## Step 2: Edit the redirection module

Within the CMB-ML repository, open (cmbml/pyilc_redir/__init__.py)[./cmbml/pyilc_redir/__init__.py]. Edit the path to match the location where you've installed PyILC, specifically `input.py` and `wavelets.py`.

After the edits (assuming the example above), it should look like:
```python
import sys
sys.path.append('/home/code/pyilc/pyilc')

from input import ILCInfo
from wavelets import Wavelets, wavelet_ILC, harmonic_ILC
```

**Note the double "pyilc" at the end.**

## Why?

This allows CMB-ML to import the required code from PyILC. While it could be run as a script, we instead run it using the defined classes. This enabled easier debugging, through what is already set up for CMB-ML.

It isn't ideal, and I don't recommend this practice in general as it may cause security vulnerabilities or path conflicts. However, for now, it's the most practical way to integrate PyILC into CMB-ML. If you have suggestions for a better approach, I'd love to hear them.

# Getting science assets

<!-- We now need to get either:
- All science assets for running simulations
- Just the asset containing the mask used for analysis -->

Now that CMB-ML knows where data needs to go, we can get it. The following table describes the different files and how they're used.

| Source | Asset | Use | Phases |
|----|----|----|----|
| Planck | NILC-Cleaned Map | Contains the mask used | Cleaning, Analysis |
| Planck | Observation Maps | Contain variance maps used for noise | Simulation | 
| WMAP   | WMAP9 Chains | Cosmological parameters for generating CMB maps | Simulation |
| Simons Observatory | planck_deltabandpass table | Concise Planck instrument details | Optional |
| CMB-ML | cmb-ml_deltabandpass table | Concise Planck-like instrument details | All |
| CMB-ML | Download records | Generating links to download the dataset | Optional |

Below are directions for getting this data. There are two methods:
- Using the automated script, download all files from the original sources (recommended)
- Manually acquiring/moving files

At the end of the process, you will have many files in your Assets directory. At the end of this section I've got an example file tree.

## Automated setup

The easiest method is the simplest: run [the get_data/get_assets.py](../get_data/get_assets.py) script. This will download from the ESA's Planck Legacy Archive and from NASA's LAMBDA Archive. Files will go to the locations specified in your configuration. Downloads may be slow.

The script uses CMB-ML code (as described in subsequent notebooks) to:
- Download Planck maps
- Download WMAP9 Chains and untar them
- Get the planck_deltabandpass.tbl
- Move the files from this repository to your assets directory

## Manual setup

If you prefer not using the script, individual files are available from the source.

- Planck Maps
    - [Planck Collaboration Observation at 30 GHz](https://irsa.ipac.caltech.edu/data/Planck/release_3/all-sky-maps/maps/LFI_SkyMap_030-BPassCorrected_1024_R3.00_full.fits)
    - [Planck Collaboration Observation at 44 GHz](https://irsa.ipac.caltech.edu/data/Planck/release_3/all-sky-maps/maps/LFI_SkyMap_044-BPassCorrected_1024_R3.00_full.fits)
    - [Planck Collaboration Observation at 70 GHz](https://irsa.ipac.caltech.edu/data/Planck/release_3/all-sky-maps/maps/LFI_SkyMap_070-BPassCorrected_1024_R3.00_full.fits)
    - [Planck Collaboration Observation at 100 GHz](https://irsa.ipac.caltech.edu/data/Planck/release_3/all-sky-maps/maps/HFI_SkyMap_100_2048_R3.01_full.fits)
    - [Planck Collaboration Observation at 143 GHz](https://irsa.ipac.caltech.edu/data/Planck/release_3/all-sky-maps/maps/HFI_SkyMap_143_2048_R3.01_full.fits)
    - [Planck Collaboration Observation at 217 GHz](https://irsa.ipac.caltech.edu/data/Planck/release_3/all-sky-maps/maps/HFI_SkyMap_217_2048_R3.01_full.fits)
    - [Planck Collaboration Observation at 353 GHz](https://irsa.ipac.caltech.edu/data/Planck/release_3/all-sky-maps/maps/HFI_SkyMap_353-psb_2048_R3.01_full.fits)
    - [Planck Collaboration Observation at 545 GHz](https://irsa.ipac.caltech.edu/data/Planck/release_3/all-sky-maps/maps/HFI_SkyMap_545_2048_R3.01_full.fits)
    - [Planck Collaboration Observation at 847 GHz](https://irsa.ipac.caltech.edu/data/Planck/release_3/all-sky-maps/maps/HFI_SkyMap_857_2048_R3.01_full.fits)
    - [Planck Collaboration NILC-cleaned Map](https://irsa.ipac.caltech.edu/data/Planck/release_3/all-sky-maps/maps/component-maps/cmb/COM_CMB_IQU-nilc_2048_R3.00_full.fits)
- Others
    - [WMAP9 Chains, direct download](https://lambda.gsfc.nasa.gov/data/map/dr5/dcp/chains/wmap_lcdm_mnu_wmap9_chains_v5.tar.gz)
    - [Planck delta bandpass table, from Simons Observatory](https://github.com/galsci/mapsims/raw/main/mapsims/data/planck_deltabandpass/planck_deltabandpass.tbl)
    - CMB-ML Assets (in this repository)

All files need to go to your assets directory (I'll assume this is simply `Assets/`).

- Planck files should go into a `Assets/Planck` subfolder. 
- The WMAP chains are downloaded as a single ".tar.gz" file. The contents of that should go into `Assets/WMAP`.
- The Planck delta bandpass table should go into `Assets/Planck` (despite being from Simons Observatory)

The CMB-ML assets are small files included in this repository. One (`cmb-ml_deltabandpass.tbl`) describes the simplified instrumentation we model in our simulations. The README gives a description of it. The others, beginning with `upload_records_` has the information needed to download the available datasets.

The whole CMB-ML folder within this repository's assets should be copied to your local system's assets.

## Science assets check

Assuming you got all files, the structure of the Assets folder should be:

```
└─ Assets
   ├─ CMB-ML
   |  ├─ cmb-ml_deltabandpass.tbl
   |  ├─ README.txt
   |  ├─ upload_records_I_128_1450.json
   |  └─ upload_records_I_512_1450.json
   ├─ Planck
   |  ├─ HFI_SkyMap_###_2048_R3.01_full.fits
   |  |  ... (### = 100, 143, 217, 353-psb, 545, 857)
   |  ├─ LFI_SkyMap_###-BPassCorrected_1024_R3.00_full.fits
   |  |  ... (### = 030, 044, 070)
   |  └─ planck_deltabandpass.tbl
   └─ WMAP
      ├─ wmap_lcdm_mnu_wmap9_chains_v5
      |   ├─ a002
      |   |  ... (many of these)
      |   └─ zstar
      └─ wmap_lcdm_mnu_wmap9_chains_v5.tar.gz
```

# Simulation Assets

It is most likely that no action is needed on your part for this section.

For simulation, foreground maps (and ancilliary information) are needed. When generating simulations, [PySM3](https://pysm3.readthedocs.io/en/latest/) relies on [astropy](https://docs.astropy.org/en/stable/index.html) to download and cache template maps. 

More information about how `astropy` does thist can be found [here](https://docs.astropy.org/en/stable/utils/data.html). The default location of maps can be set using the instructions [here](https://docs.astropy.org/en/stable/config/index.html).

Downloading templates is sometimes interrupted resulting in an error and the code crashing. It is annoying and beyond our control. However, because the templates are cached, the pipeline can be resumed without having to download any maps already completely acquired.

# Getting the dataset

**NOTE:** Having the full dataset is **not** required to run the demonstrations.

There are a few options for getting the CMB-ML dataset: it can be downloaded from Box or generated by running the simulation phase of the pipeline. Links to Box folders with individual simulations are available, but we do not recommend that route. **We strongly recommend using the included automated scripts for downloading the simulations.**

The "full" dataset is at $\mathrm{N}_\mathrm{side}=512$. There is also a lower resolution dataset at $\mathrm{N}_\mathrm{side}=128$, which may be useful for debugging code (these maps load much more quickly). **No effort has been made for accuracy of low-resolution maps**, they're simply the result of running the CMB-ML simulation phase at $\mathrm{N}_\mathrm{side}=128$.

## Automated Downloading

The `get_data.py` script will get the full-size dataset by default. The following steps can choose a different dataset or portions thereof:
- Ensure your `Assets/CMB-ML/` directory contains the `upload_records` for the dataset you want
    - Either `upload_records_I_512_1450.json` or `upload_records_I_128_1450.json`
    - Both were put there during the steps of the previous section
- In `cfg/pipeline/pipe_raw.yaml`, select the version of the shared links to use
    - The default is `upload_records_I_512_1450.json`
- In `cfg/config_sim.yaml`, choose a splits configuration
    - The default `all` will get all simulations

## Links to individual simualtions

Individual simulations can be found here:
- [Box link for CMB_ML_512_1450](https://utdallas.box.com/v/cmb-ml-512-1450)
- [Box link for CMB_ML_128_1450](https://utdallas.box.com/v/cmb-ml-128-1450)

## Generating simulations

Simulations can be generated by running `python main_sims.py` for the full size dataset, or `python main_sims.py dataset_name=CMB_ML_128_1450 nside=128` for the lower resolution dataset.

# Next steps

While following along this notebook, you should have:
- Set up your local system configuration
- Set up PyILC
- Acquired the science assets
- (Possibly) Downloaded the simulations

**Your system is now set up to use CMB-ML.**

From here, either:
- [Take a look at some simulations](./D_first_look_at_sims.ipynb)
- [Learn about the CMB-ML code](./E_CMB_ML_framework.ipynb)
- Or run any of the top-level scripts. Diving in may be the best bet, returning to the demonstrations as you need.